# Train Model

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv1D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib

def resnet_block(inputs, filters=64, kernel_size=3, strides=1):
    x = Conv1D(filters, kernel_size, padding='same', strides=strides)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv1D(filters, kernel_size, padding='same', strides=strides)(x)
    x = BatchNormalization()(x)
    
    # Add skip connection
    x = Add()([x, inputs])
    x = ReLU()(x)
    
    return x

# Connect to the SQLite database
conn = sqlite3.connect('Cleaned_Train_DB_80.db')
cursor = conn.cursor()

# Load data into df
cursor.execute("SELECT * FROM mockdata")
rows = cursor.fetchall()
columns = ['n', 'timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'label']
df = pd.DataFrame(rows, columns=columns)
conn.close()

# Preprocessing
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into features (X) and labels (y)
X = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
y = df['label']

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler to a file
joblib.dump(scaler, 'scaler.pkl')

# Split the data 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for Conv1D 
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

# Define the ResNet model
inputs = Input(shape=(X_train.shape[1], 1))
x = Conv1D(64, 3, padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)

# Add multiple ResNet blocks
for _ in range(3):
    x = resnet_block(x)

x = Conv1D(64, 3, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, 
          validation_data=(X_val, y_val), callbacks=[early_stopping])

# Save the model
model.save('ResNet_model.h5')

# Evaluate on validation set
y_pred = model.predict(X_val)
y_pred_classes = y_pred.argmax(axis=-1)

print(confusion_matrix(y_val, y_pred_classes))
print(classification_report(y_val, y_pred_classes))


Epoch 1/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7123 - loss: 0.5525 - val_accuracy: 0.7138 - val_loss: 0.5591
Epoch 2/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8020 - loss: 0.4316 - val_accuracy: 0.7964 - val_loss: 0.4327
Epoch 3/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8277 - loss: 0.3852 - val_accuracy: 0.8280 - val_loss: 0.3690
Epoch 4/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8401 - loss: 0.3615 - val_accuracy: 0.8322 - val_loss: 0.3662
Epoch 5/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8581 - loss: 0.3258 - val_accuracy: 0.8580 - val_loss: 0.3191
Epoch 6/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8748 - loss: 0.3149 - val_accuracy: 0.8459 - val_loss: 0.3395
Epoch 7/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8856 - loss: 0.2736 - val_accuracy: 0.8732 - val_loss: 0.3050
Epoch 8/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8867 - loss: 0.2707 - val_accuracy: 0.

60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
[[885  92]
 [ 67 857]]
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       977
           1       0.90      0.93      0.92       924

    accuracy                           0.92      1901
   macro avg       0.92      0.92      0.92      1901
weighted avg       0.92      0.92      0.92      1901



# Test Model on new Data

In [16]:
from tensorflow.keras.models import load_model
import pandas as pd

new_data = {
    'ax': [-0.304688],
    'ay': [0.424316],
    'az': [-0.908691],
    'gx': [-6.793893],
    'gy': [-0.725191],
    'gz': [0.618321]
}
new_df = pd.DataFrame(new_data)
model = load_model('ResNet_model.h5')
predictions = model.predict(new_df)
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))
print(predicted_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
['normal']


In [14]:
import sqlite3
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Database file 
database_file = 'testData.db'
model_file = 'ResNet_model.h5'

# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

# Fetch data from the table 
query = 'SELECT n, timestamp, ax, ay, az, gx, gy, gz FROM test_data LIMIT 100'
cursor.execute(query)
rows = cursor.fetchall()

# Convert rows to DataFrame
columns = ['n', 'timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz']
df = pd.DataFrame(rows, columns=columns)

# Preprocess the data
X = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values

# Load the scaler used during training
scaler = StandardScaler()
# Random Training Data
sample_training_data = np.random.rand(500, 6)
scaler.fit(sample_training_data)

# Normalize the features
X = scaler.transform(X)

# Reshape data for Conv1D (samples, timesteps, channels)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Load the trained model
model = load_model(model_file)

# Make predictions
y_pred = model.predict(X)
y_pred_classes = y_pred.argmax(axis=-1)

# Output results
labels = ['normal', 'anomaly']

for i, pred in enumerate(y_pred_classes):
    print(f"Timestamp: {df.iloc[i]['timestamp']} => Prediction: {labels[pred]}")

# Close the database connection
conn.close()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Timestamp: 20-05-2024 13:19:45:65564 => Prediction: normal
Timestamp: 20-05-2024 13:20:54:18225 => Prediction: normal
Timestamp: 20-05-2024 13:16:47:42277 => Prediction: normal
Timestamp: 20-05-2024 13:15:19:91737 => Prediction: normal
Timestamp: 20-05-2024 13:16:36:77520 => Prediction: normal
Timestamp: 20-05-2024 13:23:05:88818 => Prediction: normal
Timestamp: 20-05-2024 13:23:38:61220 => Prediction: normal
Timestamp: 20-05-2024 13:17:39:73074 => Prediction: normal
Timestamp: 20-05-2024 13:15:06:10220 => Prediction: normal
Timestamp: 20-05-2024 13:19:32:35887 => Prediction: normal
Timestamp: 20-05-2024 13:21:36:17714 => Prediction: normal
Timestamp: 20-05-2024 13:16:18:67092 => Prediction: normal
Timestamp: 20-05-2024 13:21:51:52998 => Prediction: normal
Timestamp: 20-05-2024 13:15:41:40409 => Prediction: normal
Timestamp: 20-05-2024 13:17:38:60793 => Prediction: normal
Timestamp: 20-05-2024 13:18:03:19136 => Prediction: normal
Timestamp: 20-05-2